In [16]:
import pandas as pd
import openai
import time
import requests
from tqdm import tqdm
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type, RetryError

#insert openAI key
openai.api_key = 'my-key'

In [17]:
# read file containing 300 randomly slected urls from the total urls 
#that we got from the search result: jerusalem, from the LOC
with open('./random_jerusalem.csv', 'r') as file:
      df = (pd.read_csv(file))

In [18]:
#convert csv to list
random_urls = df['URLs'].to_list()

In [20]:
random_urls = random_urls[:1000]

AttributeError: 'list' object has no attribute 'head'

In [ ]:
def get_selected_metadata(image_url):
    metadata_url = image_url + "?fo=json"
    try:
        response = requests.get(metadata_url)
        if response.status_code == 200:
            data = response.json()

            # Extracting nested fields from the JSON response
            interesting_fields = {
                "id": data.get("item", {}).get("id"),
                "contributor_names": data.get("item", {}).get("contributor_names"),
                "created_published": data.get("item", {}).get("created_published"),
                "created_published_date": data.get("item", {}).get("created_published_date"),
                "description": data.get("item", {}).get("description"),
                "format": data.get("item", {}).get("format"),
                "location": data.get("item", {}).get("location"),
                "title": data.get("item", {}).get("title"),
                "notes": data.get("item", {}).get("notes"),
                "number_lccn": data.get("item", {}).get("number_lccn")  # Added "number_lccn"
            }

            return interesting_fields
    except Exception as e:
        tqdm.write(f"An error occurred while fetching metadata for {image_url}: {e}")
    return None

@retry(wait=wait_exponential(multiplier=1, min=4, max=10), 
       stop=stop_after_attempt(3), 
       retry=retry_if_exception_type(openai.error.RateLimitError))
def call_openai_api(prompt, metadata, results_list):
    response = openai.ChatCompletion.create(
        model="gpt-4-1106-preview",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt.format(metadata=metadata)}
        ]
    )
    result = response.choices[0].message['content']
    results_list.append(result)
    return result

def save_results(results_list):
    with open('results.txt', 'w', encoding='utf-8') as file:
        for result in results_list:
            file.write(result + '\n')

def batch_process(urls, batch_size):
    for i in range(0, len(urls), batch_size):
        yield urls[i:i + batch_size]

def analyze_image_metadata(prompt, image_urls, batch_size=20, delay_seconds=5):
    start_time = time.time()
    results_list = []

    for batch in batch_process(image_urls, batch_size):
        for url in tqdm(batch, desc='Processing Batch', unit='image'):
            while True:
                try:
                    metadata = get_selected_metadata(url)
                    if metadata:
                        tqdm.write(f"Processing image {url}...")
                        result = call_openai_api(prompt, metadata, results_list)
                        if result:
                            tqdm.write(f"Result: {result}")
                        time.sleep(delay_seconds)  # Fixed delay between each API call
                        break  # Break the while loop once the API call is successful
                    else:
                        tqdm.write(f"No metadata available for image {url}.")
                        break  # Break the while loop if no metadata is available
                except RetryError:
                    tqdm.write("Maximum retry attempts reached. Pausing for 15 minutes before retrying.")
                    save_results(results_list)  # Save progress
                    time.sleep(900)  # Pause for 15 minutes

        # Sleep for 15 minutes between each batch
        tqdm.write("Completed a batch. Pausing for 15 minutes before starting the next batch.")
        #time.sleep(900)

    end_time = time.time()
    tqdm.write(f"Total time taken: {end_time - start_time:.2f} seconds")
    save_results(results_list)

#usage
prompt = """I am going to give you the metadata of a historical photograph. From it, extract the city and landmark(s) represented in
the picture.Your response should be in JSON format.
Don’t give any polite introduction on your response, just JSON format, following this template: 
{{
  "id": "2019696982",
  "city": "Jerusalem",
  "landmarks": ["Hebrew University"]
}}
where, id refers to a field already provided in the metadata. city refers to the city where the photo was shot. landmark refers 
to any specific building or location inside of the city.
If it is impossible to detect a specific landmark or city, use the word "Unknown".
If there are multiple landmarks depicted, list them.

Metadata: {metadata}"""  

# list of image URLs
image_urls = random_urls

# Run the analysis
analyze_image_metadata(prompt, image_urls,  delay_seconds=5)


Processing Batch:   0%|                                                                      | 0/20 [00:03<?, ?image/s]

Processing image http://www.loc.gov/item/2019698317/...


Processing Batch:   0%|                                                                      | 0/20 [00:07<?, ?image/s]

Result: ```json
{
  "id": "2019698317",
  "city": "Jerusalem",
  "landmarks": ["St. John Hostel", "German Church", "Church of Holy Sepulchre"]
}
```


Processing Batch:   5%|███                                                           | 1/20 [00:14<04:01, 12.72s/image]

...

Processing image http://www.loc.gov/item/2019697330/...


Processing Batch:  10%|██████▏                                                       | 2/20 [00:29<03:24, 11.37s/image]

Result: ```json
{
  "id": "2019697330",
  "city": "Jerusalem",
  "landmarks": ["Siloam"]
}
```


Processing Batch:  15%|█████████▎                                                    | 3/20 [00:35<03:07, 11.04s/image]

Processing image http://www.loc.gov/item/2019693918/...


In [ ]:
# To read the saved results later
with open('results.txt', 'r') as file:
    saved_results = file.readlines()
    for result in saved_results:
        print(result)